In [10]:
# Required imports
import os.path
from typing import Dict, Tuple, Optional
import torch as th
import numpy as np

# Import modules from .py files
from equations import SystemEquations, compute_derivative, SecondOrderEquations, simpson_integral, FourthOrderEquations
from model import Configuration, Model
from modules import EquationsModel
from torchinfo import summary

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import matplotlib as mpl

mpl.rc('text', usetex=True)
mpl.rc('font', family='serif')

plt.rcParams['axes.labelsize']  = 14

In [11]:
class DoubleWellEquation(SecondOrderEquations):
    """
    Physics-informed NN for the simple pendulum x'' + (x-x^3) = 0
    """
    def __init__(self, function, domain, initial_conditions, *, coeffs: tuple[float, float, float] = (1.0, 1.0, 1.0), boundary_type: str = "dirichlet"):
        super().__init__(function, domain, initial_conditions, boundary_type)
        a, b, c = coeffs
        self.coeffs = {"action": a, "dynamics": b, "hamiltonian": c}

    def configuration(self):
        config = super().configuration()
        config["coefficients"] = self.coeffs
        return config

    def equation(self, x, y, t):
        """
        Defines the second-order differential equation x'' = f(x, x', t).
        In this case, x'' = -x + x^3.
        """
        return -x + x**3

    def calculate_loss(self, inputs: th.Tensor, outputs: Dict[str, th.Tensor]) -> th.Tensor:

        trial = self.calculate_trial_solution(inputs, outputs)
        x_trial, x_dot = trial["x"], trial["y"]
        x_double_dot = compute_derivative(inputs, x_dot)
        
        # 3) residual penalties enforcing x'' = −x + x**3
        mse_d = th.nn.MSELoss(reduction='sum')
        mse_dynamics = mse_d(x_double_dot, -x_trial + x_trial**3)

        H = 0.5 * x_dot**2 + 0.5 * x_trial**2 - 0.25 * x_trial**4
        H_mean = H.mean()
        mse_hamiltonian = ((H - H_mean)**2).sum()
        
        # 4) pendulum Lagrangian L = ½ y² − (½ x² − 1/4 x^4)
        L = (0.5 * x_dot**2 - 0.5 * x_trial**2 + 0.25 * x_trial**4)
        action = th.trapz(L.view(-1), inputs.view(-1))
        #action = simpson_integral(L.view(-1), inputs.view(-1))
        
        # 5) combine (you can tune a,b if you like)
        a, b, c = self.coeffs["action"], self.coeffs["dynamics"], self.coeffs["hamiltonian"]
        total = a*action + b*mse_dynamics + c*mse_hamiltonian

        return {"total": total, "dynamics": mse_dynamics, "hamiltonian": mse_hamiltonian}

    def system(self, t, z):
        x, y = z
        dxdt = y
        dydt = -x+x**3
        return [dxdt, dydt]


configuration_dw = Configuration(
                seed=4235,
                features=[64, 64, 64, 64, 64],
                activation_function=th.nn.Tanh(),
                learning_rate=1e-5,
                epochs =0,
                steps=15_000,
            )

equations_dw = DoubleWellEquation(
                function="x",
                domain=(-7.5,7.5),
                initial_conditions={
                    "x": (-7.5, -1),
                    "y": (7.5, 1),
                },
                coeffs = (1,1,1),
                boundary_type="dirichlet"
            )


model_dw = Model("post", configuration_dw, equations_dw)

# ------------------------------------------------------------------
# 2)  Load the saved weights ---------------------------------------
# ------------------------------------------------------------------
run_dir = r"Resultados_Finais_Considerados/double-well-vFinal2/2025-06-03_11-25-18"   # <-- change!
model_dw.load(os.path.join(run_dir, "model_best.pt"),
            os.path.join(run_dir, "optimizer_best.pt"))
model_dw.model.eval()

t_grid = np.linspace(*equations_dw.domain, configuration_dw.steps)
trial_dw  = model_dw.eval(t_grid)              # {'x','y','z','w'}

# unpack & flatten
x = trial_dw["x"].ravel()
y = trial_dw["y"].ravel()

lagrangian = 0.5*y**2  - 0.5 * x**2 + 0.25 * x**4 + 0.25
action     = np.trapz(lagrangian, t_grid)

print(f"Renormalised action  A[-5,5] = {action:.6e}")




Renormalised action  A[-5,5] = 9.428094e-01


In [ ]:
class InvertedDoubleWellEquation(SecondOrderEquations):
    """
    Physics-informed NN for the simple pendulum x'' + (x-x^3) = 0
    """
    def __init__(self, function, domain, initial_conditions, *, coeffs: tuple[float, float, float] = (1.0, 1.0, 1.0), boundary_type: str = "dirichlet"):
        super().__init__(function, domain, initial_conditions, boundary_type)
        a, b, c = coeffs
        self.coeffs = {"action": a, "dynamics": b, "hamiltonian": c}

    def configuration(self):
        config = super().configuration()
        config["coefficients"] = self.coeffs
        return config

    def equation(self, x, y, t):
        """
        Defines the second-order differential equation x'' = f(x, x', t).
        In this case, x'' = x - x^3.
        """
        return x - x**3

    def calculate_loss(self, inputs: th.Tensor, outputs: Dict[str, th.Tensor]) -> th.Tensor:

        trial = self.calculate_trial_solution(inputs, outputs)
        x_trial, x_dot = trial["x"], trial["y"]
        x_double_dot = compute_derivative(inputs, x_dot)
        
        # 3) residual penalties enforcing x'' = x - x**3
        mse_d = th.nn.MSELoss(reduction='sum')
        mse_dynamics = mse_d(x_double_dot, x_trial - x_trial**3)

        H = 0.5 * x_dot**2 - 0.5 * x_trial**2 + 0.25 * x_trial**4
        H_mean = H.mean()
        mse_hamiltonian = ((H - H_mean)**2).sum()
        
        # 4) pendulum Lagrangian L = ½ y² − (½ x² − 1/4 x^4)
        L = (0.5 * x_dot**2 + 0.5 * x_trial**2 - 0.25 * x_trial**4)
        action = th.trapz(L.view(-1), inputs.view(-1))
        #action = simpson_integral(L.view(-1), inputs.view(-1))
        
        t_idx_0 = (inputs - 0).abs().argmin()
        x0_pred = x_trial[t_idx_0]
        target_x0 = (2**0.5)
        
        # 5) combine (you can tune a,b if you like)
        a, b, c = self.coeffs["action"], self.coeffs["dynamics"], self.coeffs["hamiltonian"]
        total = a*action + b*mse_dynamics + c*mse_hamiltonian + 1000*(x0_pred - target_x0)**2

        return {"total": total, "dynamics": mse_dynamics, "hamiltonian": mse_hamiltonian}

    def system(self, t, z):
        x, y = z
        dxdt = y
        dydt = x-x**3
        return [dxdt, dydt]


configuration_idw = Configuration(
                seed=4235,
                features=[64, 64, 64, 64, 64],
                activation_function=th.nn.Tanh(),
                learning_rate=1e-5,
                epochs =0,
                steps=15_000,
            )

equations_idw = InvertedDoubleWellEquation(
                function="x",
                domain=(-7.5,7.5),
                initial_conditions={
                    "x": (-7.5, 0),
                    "y": (7.5, 0),
                },
                coeffs = (1,1,1),
                boundary_type="dirichlet"
            )

model_idw = Model("post", configuration_idw, equations_idw)

# ------------------------------------------------------------------
# 2)  Load the saved weights ---------------------------------------
# ------------------------------------------------------------------

run_dir = r"Resultados_Finais_Considerados/inverted-double-well-vFinal2/2025-06-03_16-27-01"   # <-- change!
model_idw.load(os.path.join(run_dir, "model_best.pt"),
               os.path.join(run_dir, "optimizer_best.pt"))
model_idw.model.eval()

t_grid = np.linspace(*equations_idw.domain, configuration_idw.steps)
trial_idw  = model_idw.eval(t_grid)              # {'x','y','z','w'}

# unpack & flatten
x = trial_idw["x"].ravel()
y = trial_idw["y"].ravel()

lagrangian = 0.5*y**2  + 0.5 * x**2 - 0.25 * x**4
action     = np.trapz(lagrangian, t_grid)

print(f"Renormalised action  A[-5,5] = {action:.6e}")

In [ ]:
class KolmogorovConsecutive(FourthOrderEquations):

    def __init__(
        self,
        domain: Tuple[float, float],
        initial_conditions: Dict[str, Tuple[float, float]],
        boundary_type: str = "dirichlet"
    ):
        super().__init__("x", domain, initial_conditions, boundary_type="dirichlet")

    # RHS of the ODE ----------------------------------------------------
    def equation(self, x, y, z, w, t):
        return 3*np.pi*z + np.pi*np.sin(np.pi*x)

    # Action-augmented loss --------------------------------------------
    def calculate_loss(self, t: th.Tensor, nn_out: Dict[str, th.Tensor]) -> th.Tensor:

        # trial solutions & derivatives
        trial  = self.calculate_trial_solution(t, nn_out)
        x, y, z, w = trial["x"], trial["y"], trial["z"], trial["w"]
        x4 = compute_derivative(t, w) 

        mse_d = th.nn.MSELoss(reduction='sum')
        mse_dyn = mse_d(x4, 3*np.pi*z + np.pi*th.sin(np.pi*x))

        t_idx_0 = (t - 0).abs().argmin()
        x0_pred = x[t_idx_0]
        target_x0 = 0

        '''
        mask = (th.abs(t) > 2.5).view(-1)
        if mask.any():
            strip_penalty = th.relu(2.0 - th.abs(x[mask]))**2
            L_strip = strip_penalty.sum()                   # scalar
        else:
            L_strip = th.tensor(0.0, device=t.device)'''

        lagrangian = 0.5 * z**2 + (1.5)*np.pi*y**2 + 1 + th.cos(np.pi*x)
        action = th.trapz(lagrangian.view(-1), t.view(-1))

        total = mse_dyn + action + 1000000*(x0_pred - target_x0)**2

        return {"total": total, "dynamics": mse_dyn, "hamiltonian": mse_dyn}


cfg1 = Configuration(
    seed               = 43365,          # must match the training run
    features           = [16, 16, 16],
    activation_function= th.nn.Tanh(),
    learning_rate      = 1e-4,
    epochs             = 0,              # no training now
    steps              = 5000             # finer grid for the integral
)

system1 = KolmogorovConsecutive(
    domain = (-5, 5),
    initial_conditions = {
        "x": (-5, -1),
        "y": (-5,  0),
        "z": ( 5,  1),
        "w": ( 5,  0),
    }
)

model1 = Model("post", cfg1, system1)

# ------------------------------------------------------------------
# 2)  Load the saved weights ---------------------------------------
# ------------------------------------------------------------------
run_dir = r"Resultados_Finais_Considerados/KolmogorovConsecutivevFinal2/2025-06-05_08-55-04"   # <-- change!
model1.load(os.path.join(run_dir, "model_best.pt"),
            os.path.join(run_dir, "optimizer_best.pt"))
model1.model.eval()

t_grid = np.linspace(*system1.domain, cfg1.steps)
trial  = model1.eval(t_grid)              # {'x','y','z','w'}

# unpack & flatten
x = trial["x"].ravel()
y = trial["y"].ravel()
z = trial["z"].ravel()

lagrangian = 0.5*z**2 + 1.5*np.pi*y**2 + 1 + np.cos(np.pi*x)
action     = np.trapz(lagrangian, t_grid)

print(f"Renormalised action  A[-5,5] = {action:.6e}")

In [2]:
class KolmogorovNonConsecutive(FourthOrderEquations):

    def __init__(
        self,
        domain: Tuple[float, float],
        initial_conditions: Dict[str, Tuple[float, float]]
    ):
        super().__init__("x", domain, initial_conditions, boundary_type="dirichlet")

    # RHS of the ODE ----------------------------------------------------
    def equation(self, x, y, z, w, t):
        return 3*np.pi*z + np.pi*np.sin(np.pi*x)

    # Action-augmented loss --------------------------------------------
    def calculate_loss(self, t: th.Tensor, nn_out: Dict[str, th.Tensor]) -> th.Tensor:

        # trial solutions & derivatives
        trial  = self.calculate_trial_solution(t, nn_out)
        x, y, z, w = trial["x"], trial["y"], trial["z"], trial["w"]
        x4 = compute_derivative(t, w) 

        mse_d = th.nn.MSELoss(reduction='sum')
        mse_dyn = mse_d(x4, 3*np.pi*z + np.pi*th.sin(np.pi*x))

        lagrangian = 0.5 * z**2 + (1.5)*np.pi*y**2 + 1 + th.cos(np.pi*x)
        action = th.trapz(lagrangian.view(-1), t.view(-1))

        t_idx_0 = (t - 0).abs().argmin()
        x0_pred = x[t_idx_0]
        target_x0 = 0

        neg_y_penalty = th.relu(-y)**2
        L_yneg = neg_y_penalty.sum()

        total = mse_dyn + action + 1000000*(x0_pred - target_x0)**2 + 1000000*L_yneg
        
        return {"total": total, "dynamics": mse_dyn, "hamiltonian": mse_dyn}

cfg_non = Configuration(
    seed               = 43365,          # must match the training run
    features           = [16, 16, 16],
    activation_function= th.nn.Tanh(),
    learning_rate      = 5e-4,
    epochs             = 0,              # no training now
    steps              = 5000             # finer grid for the integral
)

system_non = KolmogorovNonConsecutive(
    domain = (-5, 5),
    initial_conditions = {
        "x": (-5, -3),
        "y": (-5,  0),
        "z": ( 5,  3),
        "w": ( 5,  0),
    }
)

model_non = Model("post_non", cfg_non, system_non)

# ------------------------------------------------------------------
# 2)  Load the saved weights ---------------------------------------
# ------------------------------------------------------------------

run_dir_non = r"Resultados_Finais_Considerados/fisher-kolmogorov-3T/2025-06-03_23-36-15"   # <-- change!
model_non.load(os.path.join(run_dir_non, "model_best.pt"),
            os.path.join(run_dir_non, "optimizer_best.pt"))
model_non.model.eval()

t_grid_non = np.linspace(*system_non.domain, cfg_non.steps)
trial_non  = model_non.eval(t_grid_non)              # {'x','y','z','w'}

# unpack & flatten
x_non = trial_non["x"].ravel()
y_non = trial_non["y"].ravel()
z_non = trial_non["z"].ravel()

lagrangian_non = 0.5*z_non**2 + 1.5*np.pi*y_non**2 + 1 + np.cos(np.pi*x_non)
action_non     = np.trapz(lagrangian_non, t_grid_non)

print(f"Renormalised action  A[-5,5] = {action_non:.6e}")

Renormalised action  A[-5,5] = 2.713747e+01


In [ ]:
t = np.linspace(system2.domain[0], system2.domain[1], 10_000)

pred = model2.eval(t)                 # returns a dict {"x","y","z","w"}
x, xdot, xddot, x3dot = (
    pred["x"].ravel(),
    pred["y"].ravel(),                # ẋ
    pred["z"].ravel(),                # ẍ
    pred["w"].ravel(),                # …x‴
)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].plot(t, xdot, lw=1.8)
axes[0].set_xlabel(r"$t$")
axes[0].set_ylabel(r"$\dot{x}$")

axes[1].plot(t, xddot, lw=1.8)
axes[1].set_xlabel(r"$t$")
axes[1].set_ylabel(r"$\ddot{x}$")

axes[2].plot(t, x3dot, lw=1.8)
axes[2].set_xlabel(r"$t$")
axes[2].set_ylabel(r"$\ddot{x}$")

for ax in axes:
    ax.spines["top"  ].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.tick_params(direction="in")

plt.tight_layout()
plt.savefig("xdot_x_xddot_x_x3dot_1x.png", dpi=300)
plt.show()

In [12]:
class PendulumChapter2nd(SecondOrderEquations):
    """
    Physics-informed NN for the simple pendulum x'' + sin(x) = 0,
    usingboth an action integral and a residual penalty.
    """
    def __init__(self, function, domain, initial_conditions, *, coeffs: tuple[float, float, float] = (1.0, 1.0, 1.0), boundary_type: str = "dirichlet"):
        super().__init__(function, domain, initial_conditions, boundary_type)
        a, b, c = coeffs
        self.coeffs = {"action": a, "dynamics": b, "hamiltonian": c}

    def configuration(self):
        config = super().configuration()
        config["coefficients"] = self.coeffs
        return config

    def equation(self, x, y, t):
        """
        Defines the second-order differential equation x'' = f(x, x', t).
        In this case, x'' = -sin(x).
        """
        return -np.sin(x)

    def calculate_loss(self, inputs: th.Tensor, outputs: Dict[str, th.Tensor]) -> th.Tensor:

        trial = self.calculate_trial_solution(inputs, outputs)
        x_trial, x_dot = trial["x"], trial["y"]
        x_double_dot = compute_derivative(inputs, x_dot)
        
        # 3) residual penalties enforcing x'' = −sin(x)
        mse_d = th.nn.MSELoss(reduction='sum')
        mse_dynamics = mse_d(x_double_dot, -th.sin(x_trial))

        mse_h = th.nn.MSELoss(reduction='sum')
        H = 0.5 * x_dot**2 + (1.0 - th.cos(x_trial))
        x0 = x_trial[0]
        y0 = x_dot[0]
        H0 = 0.5 * y0**2 + (1.0 - th.cos(x0))
        H0_t = H.new_ones(H.shape) * H0
        mse_hamiltonian = mse_h(H, H0_t)
        
        # 4) pendulum Lagrangian L = ½ y² − (1 − cos x)
        L = 0.5 * x_dot**2 - (1.0 - th.cos(x_trial))
        action = th.trapz(L.view(-1), inputs.view(-1))
        #action = simpson_integral(L.view(-1), inputs.view(-1))
        
        # 5) combine (you can tune a,b if you like)
        a, b, c = self.coeffs["action"], self.coeffs["dynamics"], self.coeffs["hamiltonian"]
        total = a*action + b*mse_dynamics + c*mse_hamiltonian

        return {"total": total, "dynamics": mse_dynamics, "hamiltonian": mse_hamiltonian}

    def system(self, t, z):
        x, y = z
        dxdt = y
        dydt = -np.sin(x)
        return [dxdt, dydt]

cfg_pend = Configuration(
                seed=4235,
                features=[64, 64, 64, 64, 64],
                activation_function=th.nn.Tanh(),
                learning_rate=1e-4,
                epochs =20_000,
                steps=10_000,
            )

equations_pend = PendulumChapter2nd(
                function="x",
                domain=(-10,10),
                initial_conditions={
                    "x": (-10, -np.pi),
                    "y": (10, np.pi),
                },
                coeffs = (1,1,1),
                boundary_type="dirichlet"
            )

model_pend = Model("post", cfg_pend, equations_pend)

# ------------------------------------------------------------------
# 2)  Load the saved weights ---------------------------------------
# ------------------------------------------------------------------
run_dir = r"Resultados_Finais_Considerados/simple-pendulum-heteroclinic/2025-05-30_17-14-20 - best result"   # <-- change!
model_pend.load(os.path.join(run_dir, "model_best.pt"),
            os.path.join(run_dir, "optimizer_best.pt"))
model_pend.model.eval()

t_grid = np.linspace(*equations_pend.domain, 100_000)
trial_pend  = model_pend.eval(t_grid)

# unpack & flatten
x = trial_pend["x"].ravel()
y = trial_pend["y"].ravel()

lagrangian = 0.5 * y**2 - (1.0 - np.cos(x)) + 2
action     = np.trapz(lagrangian, t_grid)

print(f"Renormalised action  A[-10,10] = {action:.6e}")

Renormalised action  A[-10,10] = 8.000001e+00
